# Synchronized tdmclient interactive session
## Interactive widgets

This notebook illustrates the use of `tdmclient.notebook` with interactive widgets provided by the `ipywidgets` package.

As usual, make sure the latest version of tdmclient is installed (you can skip this step if you did it recently):

In [ ]:
%pip install --upgrade tdmclient

Import the required classes and connect to the robot. In addition to `tdmclient.notebook`, `ipywidgets` provides support for interactive widgets, i.e. GUI elements which you can control with the mouse.

In [ ]:
import tdmclient.notebook
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
await tdmclient.notebook.start()

A function can be made interactive by adding a decorator `@interact` which specifies the range of values of each argument. When the cell is executed, sliders are displayed for each interactive argument. `(0,32,1)` means a range of integer values from 0 to 32 with a step of 1. Since the default value of the step is 1, we can just write `(0,32)`. The initial value of the arguments is given by their default value in the function definition.

Thymio variables aren't synchronized automatically when they're located inside functions. By adding a decorator `@tdmclient.notebook.sync_var`, all Thymio variables referenced in the function are fetched from the robot before the function execution and sent back to the robot afterwards. Note the order of the decorators: `@tdmclient.notebook.sync_var` modifies the function to make its variables synchronized with the robot, and `@interact` makes this modified function interactive.

In [ ]:
@interact(red=(0,32), green=(0,32), blue=(0,32))
@tdmclient.notebook.sync_var
def rgb(red=0, green=0, blue=0):
    global leds_top
    leds_top = [red, green, blue]

Here are alternative ways for the same result. Instead of a decorator in front of the function, you can call `interact` as a normal function, passing it the function whose arguments are manipulated interactively. Instead of decorating the function with `@tdmclient.notebook.sync_var`, you can call explicitly `set_var` to change the robot variables. And if your function is just a simple expression (a call to `set_var` or to another function if the values of its arguments don't fit directly the sliders of `interact`), you can replace it with a lambda expression.

In [ ]:
interact(lambda red=0,green=0,blue=0: set_var(leds_top=[red,green,blue]), red=(0,32), green=(0,32), blue=(0,32));

You can combine a program running on the robot and interactive controls in the notebook to change variables. Here is a program which uses its front proximity sensor to remain at some distance from an obstacle. Put your hand or a white box in front of the Thymio before you run the cell, or be ready to catch it before it falls off the table.

In [ ]:
%%run_python

prox0 = 1000
gain_prc = 2
timer_period[0] = 100

@onevent
def timer0():
    global prox_horizontal, motor_left_target, motor_right_target, prox0, gain_prc
    speed = math_muldiv(prox0 - prox_horizontal[2], gain_prc, 100)
    motor_left_target = speed
    motor_right_target = speed

The global variables created by the program are also synchronized with those in the notebook:

In [ ]:
prox0

In [ ]:
gain_prc = 5

Changing the value of `prox0`, which is related to the distance the robot will maintain with respect to the obstacle, can be done with a slider as for `leds_top` above:

In [ ]:
@interact(prox_target=(0, 4000, 10))
@tdmclient.notebook.sync_var
def change_prox0(prox_target):
    global prox0
    prox0 = prox_target

Change the value of the target value of the proximity sensor with the slider and observe how the robot moves backward or forward until it reaches a position where the expression `prox0 - prox_horizontal[2]` is 0, hence the speed is 0. Actually because it's unlikely the sensor reading remains perfectly constant, the robot will continue making small adjustments.

When you've finished experimenting, stop the program:

In [ ]:
stop()